In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [2]:
import json
f = open("configs/exp_01.json")
cfg = json.load(f)

In [3]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.batch_norm = nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.layer_norm = nn.LayerNorm(120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        residual = x
        x = self.pool(F.relu(self.conv1(x)))
        if cfg["use_batch_norm"] == True:
            x = self.pool(F.relu(self.batch_norm(self.conv2(x))))
        else:
            x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        if cfg["use_layernorm"] == True:
            x = F.relu(self.layer_norm(self.fc1(x)))
        else:
            x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [4]:

import torch.optim as optim


for seed in cfg["seeds"] :
    net = Net()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    # torch.manual_seed(0)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # np.random.seed(seed)


    for epoch in range(2):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0
    PATH = './models/cifar_net'+str(seed)+'.pth'
    print('Finished Training'+ str(seed))
    torch.save(net.state_dict(), PATH)

/nfs/stak/users/kokatea/hpc-share/anaconda3/envs/myEnv/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448238472/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1,  2000] loss: 1.914
[1,  4000] loss: 1.600
[1,  6000] loss: 1.513
[1,  8000] loss: 1.448
[1, 10000] loss: 1.416
[1, 12000] loss: 1.350
[2,  2000] loss: 1.307
[2,  4000] loss: 1.285
[2,  6000] loss: 1.245
[2,  8000] loss: 1.236
[2, 10000] loss: 1.232
[2, 12000] loss: 1.206
Finished Training0
[1,  2000] loss: 1.868
[1,  4000] loss: 1.629
[1,  6000] loss: 1.517
[1,  8000] loss: 1.462
[1, 10000] loss: 1.408
[1, 12000] loss: 1.367
[2,  2000] loss: 1.302
[2,  4000] loss: 1.278
[2,  6000] loss: 1.256
[2,  8000] loss: 1.246
[2, 10000] loss: 1.222
[2, 12000] loss: 1.200
Finished Training1
[1,  2000] loss: 1.918
[1,  4000] loss: 1.642
[1,  6000] loss: 1.551
[1,  8000] loss: 1.500
[1, 10000] loss: 1.457
[1, 12000] loss: 1.438
[2,  2000] loss: 1.335
[2,  4000] loss: 1.326
[2,  6000] loss: 1.323
[2,  8000] loss: 1.292
[2, 10000] loss: 1.265
[2, 12000] loss: 1.248
Finished Training2
[1,  2000] loss: 1.871
[1,  4000] loss: 1.606
[1,  6000] loss: 1.528
[1,  8000] loss: 1.474
[1, 10000] loss: 1.407


In [5]:
for seed in cfg["seeds"] :
    PATH = './models/cifar_net'+str(seed)+'.pth'
    net = Net()
    net.load_state_dict(torch.load(PATH))
    
    print("seed"+str(seed))

    # prepare to count predictions for each class
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}

    # again no gradients needed
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predictions = torch.max(outputs, 1)
            # collect the correct predictions for each class
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1


    # print accuracy for each class
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

seed0
Accuracy for class: plane is 61.7 %
Accuracy for class: car   is 64.9 %
Accuracy for class: bird  is 36.8 %
Accuracy for class: cat   is 62.2 %
Accuracy for class: deer  is 43.4 %
Accuracy for class: dog   is 35.8 %
Accuracy for class: frog  is 43.5 %
Accuracy for class: horse is 70.3 %
Accuracy for class: ship  is 69.0 %
Accuracy for class: truck is 68.2 %
seed1
Accuracy for class: plane is 67.8 %
Accuracy for class: car   is 72.0 %
Accuracy for class: bird  is 47.6 %
Accuracy for class: cat   is 24.0 %
Accuracy for class: deer  is 31.4 %
Accuracy for class: dog   is 68.4 %
Accuracy for class: frog  is 71.9 %
Accuracy for class: horse is 52.6 %
Accuracy for class: ship  is 77.2 %
Accuracy for class: truck is 57.3 %
seed2
Accuracy for class: plane is 62.1 %
Accuracy for class: car   is 72.2 %
Accuracy for class: bird  is 35.4 %
Accuracy for class: cat   is 21.2 %
Accuracy for class: deer  is 42.6 %
Accuracy for class: dog   is 73.9 %
Accuracy for class: frog  is 64.2 %
Accuracy f